In [1]:
from torch import nn
from mlip.pes import PotentialNeuralNet
from mlip.reann import REANN, compress_symbols

def gen_module(species=None, moduledict=None, modulelist=None, nmax=2, lmax=10, loop=2, rcut=6.0, device='cpu'):

    encode, decode, numbers = compress_symbols(species)
    species = list(set(numbers))
    reann = REANN(species, modulelist=modulelist, nmax=nmax, lmax=lmax, loop=loop, device=device, rcut=rcut)
    
    return PotentialNeuralNet(reann, moduledict, species)
model = gen_module(species=[29], lmax = 2, nmax = 15, loop = 2, device='cuda')

/scratch/x2419a03/.conda/envs/simple/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Nomenclature
# SPEC-PECriG(symbols, positions, energies, cells, pbcs, energyidx, crystalidx, gradients)

import torch as tc
from torch.utils.data import Dataset, DataLoader

class BPTypeDataset(Dataset):
    
    """Behler Parrinello Type datasets
    Indexing should be done in the unit of crystal, a set of atom used in one calculation.
    
    
    Parameters
    ----------
        symbols: List
        positions: List
        energies: List
        cells: List
    
    """
    def __init__(self, symbols, positions, energies, cells, pbcs, energyidx, crystalidx, gradients):
        self.symbols = symbols
        self.positions = positions
        self.energies = energies
        self.cells = cells
        self.pbcs = pbcs
        self.energyidx = energyidx
        self.crystalidx = crystalidx
        self.gradients = gradients


    def __len__(self):
        return len(self.energies)
    
    def __getitem__(self, idx):
        return self.symbols[idx], self.positions[idx], self.energies[idx], self.cells[idx], self.pbcs[idx], self.energyidx[idx], self.crystalidx[idx], self.gradients[idx]

    
def concate(batch, device='cuda'):
    cat = lambda x: tc.from_numpy(np.concatenate(x))
    
    symbols, positions, energies, cells, pbcs, energyidx, crystalidx, gradients = [], [], [], [], [], [], [], []
    for data in batch:
        symbol, position, energy, cell, pbc, energyid, crystalid, gradient = data
        symbols.append(symbol)
        positions.append(position)
        energies.append(energy)
        cells.append(cell[None])
        pbcs.append(pbc[None])      
        energyidx.append(energyid)
        crystalidx.append(crystalid)
        gradients.append(gradient)

    return (cat(symbols), cat(positions).to(device=device).requires_grad_(True), 
            tc.tensor(energies).to(device=device), cat(cells).to(device=device).requires_grad_(True), 
            cat(pbcs), tc.tensor(energyidx).to(device=device), cat(crystalidx).to(device=device), cat(gradients).to(device=device))

import numpy as np
from pymatgen.core import Structure
from monty.serialization import loadfn

def get_dataloader(location, symbol, number, name, batch_size=10):

    data = loadfn(location + symbol + '/' + name)
    encode, decode, numbers = compress_symbols([number])

    #data[0]['structure'].cart_coords;
    #data[0]['structure'].lattice.matrix;
    #data[0]['outputs']['forces'];
    #data[0]['structure'].lattice.pbc;
    #data[0]['num_atoms']
    symbols, positions, energies, cells, pbcs, energyidx, crystalidx, gradients  = [], [], [], [], [], [], [], []
    for idx, d in enumerate(data):   
#        if d['outputs']['energy'] > -400:
#            continue
        symbols.append([encode[n] for n in d['structure'].atomic_numbers])
        positions.append(d['structure'].cart_coords)
        energies.append(d['outputs']['energy'])
        cells.append(d['structure'].lattice.matrix)
        pbcs.append(np.array(d['structure'].lattice.pbc))
        energyidx.append(idx)
        crystalidx.append([idx] * data[idx]['num_atoms'])
        gradients.append(-np.array(d['outputs']['forces']))

    imgdataset = BPTypeDataset(symbols, positions, energies, cells, pbcs, energyidx, crystalidx, gradients)
    
    return imgdataset, DataLoader(imgdataset, batch_size=batch_size, shuffle=True, collate_fn=concate)

location = "/home01/x2419a03/libCalc/mlip/data/"
symbol = 'Cu'
number = 29
imgdataset, dataloader = get_dataloader(location, symbol, number, 'training.json')

In [3]:
class MSEFLoss:
    def __init__(self, muE=1., muF=1.):
        self.muE = muE
        self.muF = muF
    def __call__(self, predE, predF, y, dy):
        #_, NA = tc.unique_consecutive(crystalidx, return_counts=True)
        #self.NTA = len(dy)
        #self.lossE = tc.sqrt(tc.sum(((y - predE)/ NA)**2) / len(y))
        #self.lossG = tc.sqrt(tc.sum((predF - dy)**2) / self.NTA)
        self.lossE = tc.sum((y - predE)**2)
        self.lossG = tc.sum((dy - predF)**2)
        return self.muE * self.lossE + self.muF * self.lossG


class Normalizer(object):
    """Normalize a Tensor and restore it later. """

    def __init__(self, tensor, N, device='cuda'):
        """tensor is taken as a sample to calculate the mean and std"""
        self.mean = tc.mean(tensor).to(device=device) / N
        self.std = tc.std(tensor).to(device=device)

    def norm(self, tensor, N):
        #return (tensor - self.mean) / self.std
        return tensor - N * self.mean

    def denorm(self, normed_tensor, N):
        #return normed_tensor * self.std + self.mean
        return normed_tensor + self.mean

    def state_dict(self):
        return {'mean': self.mean,
                'std': self.std}

    def load_state_dict(self, state_dict):
        self.mean = state_dict['mean']
        self.std = state_dict['std']

In [ ]:
import torch as tc
from torch.autograd import grad

def test(dataloader, model, loss_fn, normalizer, device='cuda'):
        
    lossE, lossG, NTA = 0., 0., 0
    model.eval()
    for batch, _ in enumerate(dataloader):

        symbols, positions, energies, cells, pbcs, energyidx, crystalidx, gradients  = _
#        with torch.no_grad():
        _, pred, predG = model(symbols, positions, cells, pbcs, energyidx, crystalidx)
        A = len(gradients)
        
#        loss = loss_fn(normalizer.denorm(pred, A), predG, energies, gradients)
        loss = loss_fn(pred, predG, energies, gradients)
        
        NTA += A
        lossE += loss_fn.lossE.item()
        lossG += loss_fn.lossG.item()

    return lossE / NTA, lossG / NTA
    

def train(dataloader, model, loss_fn, optimizer, normalizer, device='cuda'):
    
    lossE, lossG, NTA = 0., 0., 0
    model.train()
    for batch, _ in enumerate(dataloader):

        symbols, positions, energies, cells, pbcs, energyidx, crystalidx, gradients  = _
        
        # Backpropagation
        optimizer.zero_grad()

        _, pred, predG = model(symbols, positions, cells, pbcs, energyidx, crystalidx)
        
        A = len(gradients)
        
        loss = loss_fn(pred, predG, energies, gradients)
#        loss = loss_fn(pred, predG, normalizer.norm(energies, A), 27416.)
        
        loss.requires_grad_(True)
        
        loss.backward()
        optimizer.step()
        
        NTA += A
        lossE += loss_fn.lossE.item()
        lossG += loss_fn.lossG.item()

    return lossE / NTA, lossG / NTA

from torch.utils.tensorboard import SummaryWriter

def run(symbol, number, moduledict=None, modulelist=None, lmax=2, nmax=15, rcut=6., loop=2, batch_size=30, 
        location="/home01/x2419a03/libCalc/mlip/data/", 
        device='cpu'):
    species = [number]
    
    log_dir = './20220804/' + symbol + '_log'
    
    model = gen_module(species=species, moduledict=moduledict, modulelist=modulelist, 
                       lmax=lmax, nmax=nmax, loop=loop, rcut=rcut, device=device)
#    model = nn.DataParallel(model)
    ####
#    model.load_state_dict(tc.load('/scratch/x2419a03/workspace/20220803/Cu_weights_100.pt'))
#    model.desc.rcut = tc.tensor([rcut]).double().to(device=device)
    ####
    
    imgdataset, dataloader = get_dataloader(location, symbol, number, 'training.json', batch_size=batch_size)
    test_imgdataset, test_dataloader = get_dataloader(location, symbol, number, 'test.json', batch_size=batch_size)


    writer = SummaryWriter(log_dir=log_dir)

    normalizer = Normalizer(tc.tensor(imgdataset.energies).double(), len(imgdataset.symbols))

    lr, muF = 1e-1, 3
    for t in range(5000):
        loss_fn = MSEFLoss(muF=muF)
        lossE, lossG = train(dataloader, model, loss_fn, 
                             tc.optim.Adam(model.parameters(), lr=lr), normalizer)
        if t % 10 == 0:
            tc.save(model.state_dict(), './20220804/' + symbol + '_weights_%d.pt' % t)
        loss = lossE + lossG
        

        # 30 meV
        if loss < 3e-2:
            lr = 1e-5
            muF = 1
        # 100 meV
        elif loss < 1e-2:
            lr = 1e-4
            muF = 1.5
        # 300 meV
        elif loss < 3e-1:
            lr = 1e-3
            muF = 2
        # 1 eV
        elif loss < 1:
            lr = 1e-2
            muF = 2.5
            
#         # 30 meV
#         if lossG < 3e-2:
#             muF = 1
#         # 100 meV
#         if lossG < 1e-2:
#             muF = 3
#         # 300 meV
#         elif lossG < 3e-2:
#             muF = 10
#         # 1 eV meV
#         elif lossG < 1e-1:
#             muF = 30

        writer.add_scalar('training RMSE-E (eV/atom)', lossE, t)
        writer.add_scalar('training RMSE-F (eV/A)', lossG, t)
        
        test_lossE, test_lossG = test(test_dataloader, model, loss_fn, normalizer)
        writer.add_scalar('test RMSE-E (eV/atom)', test_lossE, t)
        writer.add_scalar('test RMSE-F (eV/A)', test_lossG, t)
        
        print(t, ": {0:<10.2f} {1:<10.2f} {2:<10.2f} {3:<10.2f}".format(lossE * 1000, lossG* 1000, test_lossE* 1000, test_lossG* 1000))

    writer.flush()
    writer.close()
    tc.cuda.empty_cache()
    
device = 'cuda'
#tables = {'Cu': 29, 'Ge': 32, 'Li': 3, 'Mo': 42, 'Ni':28, 'Si': 14}
tables = {'Cu': 29}


from mlip.reann import Gj

device='cuda'
desc_NO = 4
species = [0]
nmax = 20
loop = 3
rcut = 3.

moduledict = nn.ModuleDict()
for spe in species:
    moduledict[str(spe)] = nn.Sequential(
        nn.Linear(desc_NO, 128),
        nn.Softplus(),
        nn.Linear(128, 1),
    )
    
modulelist = nn.ModuleList()
for j in range(loop):
    descdict = nn.ModuleDict()
    for spe in species:
        descdict[str(spe)] = nn.Sequential(
                nn.Linear(desc_NO, 128),
                 nn.Softplus(),
                nn.Linear(128, nmax)
        )
    modulelist.append(Gj(descdict, species=species, nmax=nmax))

for sym, num in tables.items():
    run(sym, num, device=device, nmax=nmax, loop=loop, rcut=rcut,
        moduledict=moduledict.double().to(device=device),
        modulelist=modulelist.double().to(device=device))

0 : 1823423.38 589139.34  205110.91  53293.34  
1 : 1243499.25 10172.38   993286.02  2279.52   
2 : 503826.81  2447.39    339434.77  2255.62   
3 : 397229.08  2705.38    269910.56  5069.81   


Hello I am JaeHwan shim from jaejun Yu's group. 

I am going to explain about the Recursive embdding atomic nerual network. Recursive embedding neural network is neural network interatomic potential model which has foreground on the physics. 

Contents of my presentation is

1. REANN 적용 현황

single atoms



Results 

Results and discussion 



In [ ]:
'Cu': 29, 'Ge': 32, 'Li': 3, 'Mo': 42, 'Ni':28, 'Si': 14